In [1]:
import pandas as pd
from pathlib import Path

t = Path("./data/raw/twitter.json")
s = Path("./data/raw/SETimes.json")
t = pd.read_json(str(t))
s = pd.read_json(str(s))



In [2]:
# Remove montenegrin tweets:
t = t[~t.language.eq("me")]
t["text"] = t.tweets.apply(lambda l: " ".join(l))


In [3]:
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_columns', None)

t["source"] = "Twitter"
s["source"] = "SETimes"
ts = pd.concat([t, s])
ts = ts.drop(columns=["user", "tweets"])
url_pattern = r"[A-Za-z]+:\/\/[A-Za-z0-9\-_]+\.[A-Za-z0-9\-_:%&;\?\#\/.=]+"
username_pattern = r"@\S+"
hashtag_pattern = r"#\S+"

from utils import transliterate
ts["text"] = ts.text.str.replace(
    url_pattern, "", regex=True).str.replace(
    username_pattern, " ", regex=True).str.replace(
    hashtag_pattern, "", regex=True ).str.replace(
    "\sRT\s+:", " ", regex=True).apply(transliterate).str.replace(
    "\n", " ")
ts[ts.source=="Twitter"].sample(10)

,language,split,text,source
435,sr,dev,"vidi, zaprosi je na boks meču, i mene sad zanima da li sad ona zaprošena nastavlja da gleda boks ili idu kući da se bembaju RT sutra ću da prijavim sve ispite za avgust jer optimizam problem sa ovim sakrivanjem piva po kući je što ne može da se ohladi iza zavese tako kad me neko koga ne želim ...",Twitter
456,hr,dev,Always have your stuff when you need it with Sign up for free! FBI nadzire Android uređaje I love Dropbox because sharing razmijenio vezu Bujsko pivo Bijela kuća u subotu je priopćila da posljednje sjevernokorejske prijetnje oružjem uzima ozbiljno... Oni znaju koliko boca trebaju skupiti...,Twitter
466,bs,dev,"ja nisam kao drugi. vidim na fb svi čestitaju, ali znaš da sam uvijek bio korak ispred drugih. uredniče moj dobri :) sretan ti rođendan bivši uredniče Mrtav sam čovjek, ali mi to niko još nije saopćio. E to ja nazivam stabilizacijom! Kakva crna EU, da nam je svima zadržat status ""stabiln...",Twitter
244,sr,train,"Jedno je kad napišem ’ok’ a drugo je ’okej’ RT Zatvorim oci i kazem tiho sebi proci ce. NARAVNO DA TO NE POMOGNE I BACI ME JOS VISE U BEDAK, ali sve prodje, SUPICKUMATERI… Neću da spavam dok ne dobijem burek. RT još jedan petak provedoh sedeći kući, baš lepo U Jagodini jedino KUČKA KOJI JE V...",Twitter
364,sr,train,"Laku noć Nisam gledala Rebelde ili Internat,ja izrod+ HAHAHAHAHA VIDELA SAM SLIKU.Jaaaadna :( Znam da me volite :-* Već peta ribica koja je obula uzasne plave ugg do kolena+++ E'o mama i mamina drugarica otisle da popuse cigarete,dok ih ja ovde cekam i tripujem da ce da me ukrade neko hahaha...",Twitter
104,sr,train,"THIS JUST IN: Smurfs love to love Smurfette! Tap once a day to earn extra XP. Brainy Smurf: He's good to go to for advice, while the other Smurfs are too busy working FREE: Bag of fresh smurfberries. Smurfs only. Meet in the forest. Come alone - G.Mel Moj Mali Poni PJC-Kraljica Kris...",Twitter
108,sr,train,"Dozivao sam ke$ i on mi se javlja!!! Covek iz Iraka :D Inace ovo je napisao da se zna! :D ALEKSA NAJBOLJI SI BURAZ!! VOLIM TE ! GENIJALNO! jedna hoce vezu, druga hoce da mi pusi Dzejms Bond Filmovi se gledaju samo u HD rezoluciji Cak Noris jede hleb od sutra... hahahaha evakuacija if ...",Twitter
264,sr,train,"Pre će more do mene nego ja do mora. Mogu da spavam, a ne spavam. A deca u Africi ustaju u 6.. RT Ovi “imaš mi se“ muškarci i “imaš mi se“ žene da se spoje i da završimo s tim glupim tvitovima, ko je za, neko protiv? USV… Imajte se u tri lepe. Nije tačno, klitoris je reč turskog porekla, nas...",Twitter
582,sr,test,"ma da li si videla moj amerika album i nazvala svoj isto tako kurvo glupa ugh vintage cruise hot tattoos baby blues make me bruise smejem se već četiri reinkarnacije koliko je ovo tačno RT drkas na nekog 100 godina , vidis da prica engleski kao bugarska izbeglica, zasijes picku bila sam na c...",Twitter
143,sr,train,"RT This is for bicycles what Tesla is for cars Sleep 'boosts brain cell numbers' probaj screenflow. First Open Source Airplane Could Cost Just $15,000 | Autopia | Hey, I've compiled phalcon.so for the latest version of and PHP 5.4.10 RT CONFIRMED: FBI uses spyware to 'remotely act...",Twitter


In [4]:
# Check the composition of the dataset:
ts.groupby(by=["source", "split", "language"]).count()

text
source  split language      
SETimes dev   bs         327
              hr         327
              sr         309
        test  bs         312
              hr         313
              sr         296
        train bs        2556
              hr        2556
              sr        2262
Twitter dev   bs          15
              hr          18
              sr          79
        test  bs          15
              hr          18
              sr          79
        train bs          45
              hr          53
              sr         236

In [7]:
ts.to_json("data/interim/SETimes_and_twitter_preprocessed.jsonl", orient="records", lines=True)

In [8]:
# Shuffle the rows:
ts = ts.sample(frac=1)

In [9]:
# Format the dataset in fasttext format
for split in "test,dev,train".split(","):
    subset = ts[ts.split==split]
    content = ""
    for i, row in subset.iterrows():
        content += f"__label__{row['language']} {row['text']}\n"
    with open(
        f"data/interim/SETimes_and_Twitter_{split}.fasttext", "w") as f:
        f.write(content)
    for source in "Twitter SETimes".split():
        subset = ts[(ts.split==split) & (ts.source==source)]
        content = ""
        for i, row in subset.iterrows():
            content += f"__label__{row['language']} {row['text']}\n"
        with open(
            f"data/interim/{source}_{split}.fasttext", "w") as f:
            f.write(content)